# Predict NBA Games Tutorial

The following is an exploratory notebook for predicting the results of historial NBA games. This notebook follows the tutorial by Vikas Paruchuri of Dataquest, which can be found [here](https://www.youtube.com/watch?v=egTylm6C2is) and in [this repo](https://github.com/dataquestio/project-walkthroughs/tree/master/nba_games).